In this notebook, I make dataframes containing the white pixel locations in each floor image. I wanted just the hallway areas, but I am having trouble creating a contour around the building. I use these dataframes to push predictions into the hallway areas (hopefully) in my notebook https://www.kaggle.com/therocket290/indoor-navigation-push-to-hallway-post-process.

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from pathlib import Path
import glob
import pickle
import matplotlib.pyplot as plt
import joblib

import random
import os
import glob
import json

import cv2
from PIL import Image

In [ ]:
sites = []
for path in os.listdir('../input/wifi-features-all'):
    if (len(path.split('_')[0])>20) and (not path.split('_')[0] in sites):
        sites.append(path.split('_')[0])

In [ ]:
thresh_path = '../input/threshold-images/'
meta_path = '../input/indoor-location-navigation/metadata/'

## Example

In [ ]:
# Plot the training Data For an example Floor
example_site = '5dbc1d84c1eb61796cf7c010'
example_floorNo = 'F7'
floors = os.listdir(meta_path+example_site)
array = joblib.load(thresh_path+'thresh_list_'+example_site+'.pkl')
array = array[floors.index(example_floorNo)]
white_area = []
for (x,y) in zip(np.where(array[:,:,1] == 0)[0],np.where(array[:,:,1] == 0)[1]):
    white_area.append( (x, y) )
df = pd.DataFrame(white_area, columns=['x','y'])
df.plot(x='x', y='y', kind='scatter')

I will work on dropping the pixels outside the building.

In [ ]:
del array, white_area

In [ ]:
base="../input/indoor-location-navigation"

In [ ]:
for site in sites:
    print(site)
    site_white_areas = []
    floors = os.listdir(meta_path+site)
    array = joblib.load(thresh_path+'thresh_list_'+site+'.pkl')
    print(site+ ' ' , floors)
    for floor in floors:
        floor_array = array[floors.index(floor)]
        floor_white_area = []
        for (x,y) in zip(np.where(floor_array[:,:,1] == 0)[0],np.where(floor_array[:,:,1] == 0)[1]):
            floor_white_area.append( (x, y) )
        site_white_areas.append(floor_white_area)
        floor_df = pd.DataFrame(floor_white_area)
        floor_df.columns = ['x','y']
        
        ######
        floor_df['x2'] = floor_array.shape[0] - floor_df['x']
        
        json_plan_filename = f"{base}/metadata/{site}/{floor}/floor_info.json"
        with open(json_plan_filename) as json_file:
            json_data = json.load(json_file)

        width_meter = json_data["map_info"]["width"]
        height_meter = json_data["map_info"]["height"]
        
        # Convert to meters
        floor_df['x2'] = floor_df['x2'] * height_meter / floor_array.shape[0]
        floor_df['y2'] = floor_df['y'] * width_meter / floor_array.shape[1]
        
        #######
        floor_df.to_csv('white_area_'+site+'_'+floor+'.csv', index=False)